# Define and map land surface phenology metrics across Australia

Including the trends in the phenometrics

**To Do:**
* ~~Do some testing on running phenometrics per-pixel. Could look to loop pixel by pixel and parallise same as [hdstats](https://github.com/daleroberts/hdstats/blob/main/hdstats/ts.pyx)~~
* Sample IBRA regions to see if phenology code works well with the smaller regions
* ~~Improve ecoregions by intersecting cropping classes with ecoregions~~
* Multiprocess the phenology extraction
* ~~Multiprocess time-series extraction~~
* ~~Instead of removing the second peak, keep the highest/lowest peak of the two~~
* Consider implementing decimal time rather than year+DOY
* trends aren't per year....improve
* Validate against:
    * flux towers,
    * VODCAv2,
    * TERN phenology
    * PKU-GIMMS.
    * MODIS only
* ~~Map showing number of seasons in the 40 years~~
* Inspect how the transition between AVHRR and MODIS impacts trends in seasonal cycle for a sample of ecoregions. And the gapfilling.

In [ ]:
%matplotlib inline

import sys
import pickle
import warnings
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sb
from scipy import stats
import geopandas as gpd
import contextily as ctx
import pymannkendall as mk
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
from scipy.interpolate import griddata

sys.path.append('/g/data/os22/chad_tmp/Aus_phenology/src')
from phenology import extract_peaks_troughs, phenometrics

%load_ext autoreload
%autoreload 2

## Analysis Parameters


In [ ]:
# timeseries_file = '/g/data/os22/chad_tmp/Aus_phenology/data/ecoregions_NDVI_timeseries.pkl'
# timeseries_file ='/g/data/os22/chad_tmp/Aus_phenology/data/IBRA_regions_NDVI_timeseries.pkl'
timeseries_file ='/g/data/os22/chad_tmp/Aus_phenology/data/IBRA_subregions_NDVI_timeseries.pkl'

# ecoregions_file = '/g/data/os22/chad_tmp/Aus_phenology/data/vectors/Ecoregions2017_modified.geojson'
# ecoregions_file = '/g/data/os22/chad_tmp/Aus_phenology/data/vectors/IBRAv7_regions_modified.geojson'
ecoregions_file = '/g/data/os22/chad_tmp/Aus_phenology/data/vectors/IBRAv7_subregions_modified.geojson'

# var='ECO_NAME'
# var='REG_NAME_7'
var='SUB_NAME_7'

# region_type = 'ecoregions'
# region_type = 'IBRA_regions'
region_type = 'IBRA_subregions'

## Open data

In [ ]:
with open(timeseries_file, 'rb') as f:
    results = pickle.load(f)

gdf = gpd.read_file(ecoregions_file)

## Extract phenometrics 

<!-- import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def double_logistic_function(t, wNDVI, mNDVI, S, A, mS, mA):
    sigmoid1 = 1 / (1 + np.exp(-mS * (t - S)))
    sigmoid2 = 1 / (1 + np.exp(mA * (t - A)))
    seasonal_term = sigmoid1 + sigmoid2 - 1
    return wNDVI + (mNDVI - wNDVI) * seasonal_term

def weight_function(t, S, A, r):
    tr = 100 * (t - S) / (A - S)
    tr = np.clip(tr, 0, 100)
    return np.exp(-np.abs(r) / (1 + tr / 10))

def fit_curve(t, ndvi_observed):
    initial_guess = [np.min(ndvi_observed), np.max(ndvi_observed), np.mean(t), np.mean(t), 1, 1]
    params, _ = curve_fit(double_logistic_function, t, ndvi_observed, p0=initial_guess, maxfev=10000)
    residuals = ndvi_observed - double_logistic_function(t, *params)
    weights = weight_function(t, params[2], params[3], residuals)
    params, _ = curve_fit(double_logistic_function, t, ndvi_observed, p0=initial_guess, sigma=weights, maxfev=10000)
    return params

doys = ndvi_cycle.time.dt.dayofyear.values[2:]
doys_frac = doys/365
values = ndvi_cycle.values[2:]

##Fit the curve
parameters = fit_curve(doys_frac, values)

##Plot the observed NDVI values
plt.scatter(doys, values, label='Observed NDVI')

##Generate points for the fitted curve
t_fit = np.linspace(min(doys_frac), max(doys_frac), 365)
ndvi_fit = double_logistic_function(t_fit, *parameters)

##Plot the fitted curve
plt.plot(t_fit*365, ndvi_fit, label='Fitted Curve', color='red')

plt.xlabel('Day of the Year')
plt.ylabel('NDVI')
plt.legend()
plt.title('Double Logistic Curve Fitting for NDVI Observations')
plt.show() -->

In [ ]:
eco_regions_phenometrics = phenometrics(results,
                                        rolling=90,
                                        distance=90,
                                        prominence='auto',
                                        plateau_size=10,
                                        amplitude=0.20
                                       )

## Save results

In [ ]:
# with open(save_file, 'wb') as f:
#     pickle.dump(results, f)

## Plot one ecoregion's phenometrics

In [ ]:
l = list(results.keys())
l.sort()
l[0:5]

In [ ]:
# pheno_means[pheno_means[var]=='Northern Jarrah Forest irrigated crop'].explore(column='mean_SOS')

In [ ]:
# pheno_means[['mean_SOS', 'geometry', var]].explore(column='mean_SOS', cmap='twilight')

In [ ]:
# pheno_means[['mean_SOS', 'geometry', var]].explore(column='mean_SOS', cmap='twilight')

In [ ]:
# k='Oberon rainfed crop'
# eco_regions_phenometrics[k][['SOS_year','SOS','POS','EOS_year','EOS', 'LOS']]

In [ ]:
# eco_regions_phenometrics[k].POS.quantile(0.5)
eco_regions_phenometrics[k][['SOS','POS','EOS', 'LOS']].quantile(q=0.5, interpolation='lower')

In [ ]:
# # k='Australian Alps montane grasslands'
# k='Rainfed crop savanna'
# # k='Great Sandy-Tanami desert'
# k='Eastern Australian warm temperate forests'
# k='Southern Downs'
# k = 'Eastern Darling Downs rainfed crop'
k='Castlereagh-Barwon rainfed crop'
# k='Brigalow Belt South'
with plt.style.context('ggplot'):
    fig,ax = plt.subplots(1,1, figsize=(15,4))
    
    results[k].plot(ax=ax, c='tab:blue')
    ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].SOS.values, eco_regions_phenometrics[k].SOS_year.values)],
               y=eco_regions_phenometrics[k].vSOS,
              c='tab:green', label='SOS')
    
    ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].EOS.values, eco_regions_phenometrics[k].EOS_year.values)],
               y=eco_regions_phenometrics[k].vEOS,
              c='tab:purple', label='EOS')
    
    ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].POS.values, eco_regions_phenometrics[k].POS_year.values)],
                   y=eco_regions_phenometrics[k].vPOS,
                  c='black', label='POS')
        
    ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].TOS.values, eco_regions_phenometrics[k].TOS_year.values)],
               y=eco_regions_phenometrics[k].vTOS,
              c='tab:orange', label='TOS')
    
    ax.set_xlabel(None)
    ax.set_ylabel('NDVI')
    ax.set_title(k, fontsize=15)
    ax.legend()
    plt.tight_layout()

## Plot all regions

In [ ]:
# with plt.style.context('ggplot'):
#     fig,axes = plt.subplots(len(gdf),1, figsize=(11,65), sharex=True, layout='constrained')
#     for k,ax in zip(eco_regions_phenometrics.keys(), axes.ravel()):
    
#         results[k].plot(ax=ax)
#         ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].SOS.values, eco_regions_phenometrics[k].SOS_year.values)],
#                    y=eco_regions_phenometrics[k].vSOS,
#                   c='tab:green', label='SOS')
        
#         ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].EOS.values, eco_regions_phenometrics[k].EOS_year.values)],
#                    y=eco_regions_phenometrics[k].vEOS,
#                   c='tab:purple', label='EOS')
        
#         ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].POS.values, eco_regions_phenometrics[k].POS_year.values)],
#                        y=eco_regions_phenometrics[k].vPOS,
#                       c='black', label='POS')
            
#         ax.scatter(x=[pd.to_datetime(d-1, unit='D', origin=str(int(y))) for d,y in zip(eco_regions_phenometrics[k].TOS.values, eco_regions_phenometrics[k].TOS_year.values)],
#                    y=eco_regions_phenometrics[k].vTOS,
#                   c='tab:blue', label='TOS')
        
#         ax.set_xlabel(None)
#         ax.set_ylabel('NDVI')
#         ax.set_title(k, fontsize=8)
        # ax.legend()
    

## Summarise average phenometrics with maps of Australia

In [ ]:
means = []
for k in eco_regions_phenometrics.keys():
    df = eco_regions_phenometrics[k]

    #include a column counting the number of seasons
    df['n_seasons'] = len(df)
    
    mean_df = df.quantile(q=0.5, interpolation='lower') # median
    mean_df = mean_df.rename({i:'mean_'+i for i in mean_df.index}).to_frame().transpose()
    mean_df[var] = k

    mean_gdf = gdf.merge(mean_df, on=var)
    means.append(mean_gdf)
    
pheno_means = pd.concat(means).reset_index(drop=True)

### Plot

In [ ]:
fig,axes=plt.subplots(4,3, figsize=(11,11), sharey=True, sharex=True, layout='constrained')
pheno_stats=['SOS','vSOS', 'LOS', 
             'POS', 'vPOS', 'ROG',
             'EOS', 'vEOS', 'ROS',
             'IOS', 'vTOS', 'AOS',
      ]
cmaps = ['twilight', 'gist_earth_r', 'viridis',
         'twilight', 'gist_earth_r','magma',
         'twilight', 'gist_earth_r', 'magma_r',
         'inferno', 'gist_earth_r','plasma'
        ]
for ax,pheno,cmap in zip(axes.ravel(), pheno_stats, cmaps):
    if "v" not in pheno:
        vmin, vmax=0, 365
        label='DOY'
    if "v" in pheno:
        vmin,vmax=0.1, 0.85
        label='NDVI'
    if 'LOS' in pheno:
        vmin, vmax=160, 300
        label='days'
    if 'AOS' in pheno:
        vmin, vmax=0.05, 0.4
        label='NDVI'
    if 'IOS' in pheno:
        vmin, vmax=20, 200
        label='NDVI/\n season'
    if 'ROG' in pheno:
        vmin, vmax=0.00025, 0.0025
        label='NDVI/\nday'
    if 'ROS' in pheno:
        vmin, vmax=-0.0025, -0.00025
        label='NDVI/\nday'
    im=pheno_means.plot(column='mean_'+pheno, ax=ax, legend=False, cmap=cmap, vmin=vmin, vmax=vmax)
    ctx.add_basemap(im, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    #need to create colorbar manually to have label on top
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    ax_cbar = fig.colorbar(cbar, ax=ax, shrink=0.7)
    ax_cbar.ax.set_title(label, fontsize=8)
    ax.set_title(f'{pheno}, 1982-2021')

fig.savefig('/g/data/os22/chad_tmp/Aus_phenology/results/figs/average_phenometrics_map_'+region_type+'.png',
            bbox_inches='tight', dpi=300)

## Plot number of seasonal cycles

In [ ]:
vmin,vmax=33,45
cmap='RdYlBu'
fig,ax=plt.subplots(1,1, figsize=(8,8), layout='constrained')
im=pheno_means.plot(column='mean_n_seasons',  legend=False, cmap=cmap, vmin=vmin, vmax=vmax, ax=ax)
ctx.add_basemap(im, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
ax.axes.get_xaxis().set_ticks([])
ax.axes.get_yaxis().set_ticks([])
#need to create colorbar manually to have label on top
norm = colors.Normalize(vmin=vmin, vmax=vmax)
cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
ax_cbar = fig.colorbar(cbar, ax=ax, shrink=0.7)
ax_cbar.ax.set_title('num seasons', fontsize=8)
ax.set_title(f'Number of full vegetation cycles identified (1982-2021)')

fig.savefig('/g/data/os22/chad_tmp/Aus_phenology/results/figs/num_seasons_'+region_type+'.png',
            bbox_inches='tight', dpi=300);

## Extract robust trends


In [ ]:
trends_dfs = []
pheno_stats=['SOS','vSOS','LOS','POS','vPOS','AOS',
             'EOS', 'vTOS','vEOS','ROG','ROS', 'IOS']
i=0
for k in eco_regions_phenometrics.keys():
    print("Feature {:02}/{:02}\r".format(i + 1, len(range(0, len(eco_regions_phenometrics)))), end="")
    
    df = eco_regions_phenometrics[k]
    trends = {}
    for pheno in pheno_stats:
        
        result = mk.original_test(eco_regions_phenometrics[k][pheno], alpha=0.1)
        # result = stats.linregress(x=df.index, y=df[pheno])
        trends[pheno+'_p-value'] = result.p
        trends[pheno+'_slope'] = result.slope
        trends[pheno+'_intercept'] = result.intercept
        trends[var] = k

    trend_df=pd.DataFrame(trends, index=range(0,1))
    trends_dfs.append(gdf.merge(trend_df, on=var))
    i+=1

pheno_trends = pd.concat(trends_dfs).reset_index(drop=True)

### Plot

> Warning, this takes a very long time. I think due to the hatch plotting

In [ ]:
fig,axes=plt.subplots(4,3, figsize=(11,11),  layout='constrained')#sharey=True, sharex=True,
pheno_stats=['SOS','vSOS', 'LOS', 
             'POS', 'vPOS', 'ROG',
             'EOS', 'vEOS', 'ROS',
             'AOS', 'vTOS', 'IOS'
      ]
cmaps = ['coolwarm', 'BrBG', 'PuOr',
         'coolwarm', 'BrBG', 'Spectral',
         'coolwarm', 'BrBG', 'Spectral_r',
         'PiYG','BrBG','PuOr'
        ]
for ax,pheno,cmap in zip(axes.ravel(), pheno_stats, cmaps):
   
    if "v" not in pheno:
        vmin, vmax=-1.5,1.5
        label='days/\nyear'
    if "v" in pheno:
        vmin,vmax=-0.0015, 0.0015
        label='NDVI/\nyear'
    if 'LOS' in pheno:
        vmin, vmax=-1.5, 1.5
        label='days/\nyear'
    if 'AOS' in pheno:
        vmin, vmax=-0.002, 0.002
        label='NDVI\nyear'
    if 'ROG' in pheno:
        vmin, vmax=-2.0e-05, 2.0e-05
        label='NDVI/day/\nyear'
    if 'ROS' in pheno:
        vmin, vmax=-2.0e-05, 2.0e-05
        label='NDVI/day/\nyear'
    if 'IOS' in pheno:
        vmin, vmax=-0.4, 0.4
        label='NDVI/\nseason/\nyear'

    # im=pheno_trends.plot(column=pheno+'_slope', ax=ax, legend=False, cmap=cmap, vmin=vmin, vmax=vmax)
    
    significant = pheno_trends[pheno_trends[pheno+'_p-value'] <= 0.1]
    not_significant = pheno_trends[pheno_trends[pheno+'_p-value'] > 0.1]
    
    im=significant.plot(column=pheno+'_slope', ax=ax, legend=False, cmap=cmap, vmin=vmin, vmax=vmax, hatch='....')     
    not_significant.plot(column=pheno+'_slope', ax=ax, legend=False, cmap=cmap, vmin=vmin, vmax=vmax)     

    ctx.add_basemap(im, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    
    #need to create colorbar manually to have label on top
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    ax_cbar = fig.colorbar(cbar, ax=ax, shrink=0.7)
    ax_cbar.ax.set_title(label, fontsize=8)
    ax.set_title(f'{pheno}, 1982-2022')
    print(pheno)
    
# axes[3,2].axis('off');
fig.savefig('/g/data/os22/chad_tmp/Aus_phenology/results/figs/trends_phenometrics_map_'+region_type+'.png',
            bbox_inches='tight', dpi=300)

In [ ]:
# fig,axes=plt.subplots(1,1, figsize=(11,11),  layout='constrained')
# significant.plot(column='IOS_slope',ax=ax, legend=False, cmap='coolwarm', vmin=-1.5, vmax=1.5, hatch='XXX')   
# not_significant.plot(column='SOS_slope',ax=ax, legend=False, cmap='coolwarm', vmin=-1.5, vmax=1.5);

## Facet plots of phenology changes through time

Maybe instead group the plots by similar land covers e.g. Rainfed cropping regions, forested regions, grasslands

### Gridded phenology

In [ ]:
#create a grid
xi = np.linspace(1, 12, 12) #months
yi = np.linspace(1982, 2022, 41) #years
Xi, Yi = np.meshgrid(xi, yi) #grid coords

with plt.style.context('default'):
    fig, axes = plt.subplots(7,7, figsize=(12,13), sharex=True, sharey=True)
    for ax, (k,v) in zip(axes.reshape(-1), results.items()):
        dss = v.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean()
        zi = griddata(points=(dss.time.dt.month, dss.time.dt.year),
                      values=dss.values,
                      xi=(xi[None, :], yi[:, None]),
                      method='nearest')
        
        sp = ax.contourf(zi, levels=75, cmap='gist_earth_r')
        # clb = fig.colorbar(sp, ax=axes, shrink=0.65, format="%0.2f")
        # clb.ax.set_title('NDVI', fontsize=5)
        ax.set(xticks=range(0,12))
        ax.set_xticklabels(["J","F","M","A","M","J","J","A","S","O","N","D"], fontsize=8);
        ax.set(yticks=range(0,41))
        ax.set_yticklabels([str(y) for y in range(1982,2023)], fontsize=8);
        ax.yaxis.set_major_locator(ticker.MultipleLocator(base=10))
        ax.set_title(k, fontsize=6);
    plt.tight_layout();

fig.savefig('/g/data/os22/chad_tmp/Aus_phenology/results/figs/phenology_grids_'+region_type+'.png',
          bbox_inches='tight', dpi=300);

### Before/after seasonal curves

The better way to do this would be to recreate the seasonal cycles using the phenology stats, then we could have time-series that extend beyong just the year

In [ ]:
plt.style.use('ggplot')
fig, axes = plt.subplots(7,7, figsize=(15,13), sharex=True)
for ax, (k,v) in zip(axes.reshape(-1), results.items()):
    early_clim = v.sel(time=slice('1982', '2000')).groupby('time.month').mean()
    late_clim = v.sel(time=slice('2001', '2021')).groupby('time.month').mean()
    early_clim.plot(label='1982-2001', ax=ax)
    late_clim.plot(label='2002-2021', ax=ax)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    #ax.legend()
    ax.set_title(k, fontsize=6)
    ax.set(xticks=range(1,13))
    ax.tick_params(axis='y', labelsize=8)
    ax.set_xticklabels(["J","F","M","A","M","J","J","A","S","O","N","D"], fontsize=8);
fig.supylabel('NDVI')
plt.tight_layout()

fig.savefig('/g/data/os22/chad_tmp/Aus_phenology/results/figs/phenology_curves_'+region_type+'.png',
          bbox_inches='tight', dpi=300);